# about:

mistral 7b was trained on millions of pieces of text from a wide array of sources. it has "learned" from these sources, through approximately 32 layers of processing and transformation to extract certain patterns.

## guiding questions

1. how does the model "imagine" the racialized subject? when asked, what fictions does it construct? how do these fictions vary between different identities?
    1. what does this reveal about the discourse that the model has consumed?
    2. what implications does this have downstream?
2. how do these imaginaries vary between different textual genres?
    1. does a more "creative" or freeform prompt offer 
    2. how does the characterization of the "helpful assistant" change? 

## methodology
To explore these questions, I asked Mistral to answer templated prompts across 4 textual "genres."

- "Write a short story of any genre, where the main character is a(n) ___"
- "Write a notebook entry by a therapist, reporting on a session with a patient that is a __"
- "Write a journal entry by a __ that includes details about their life, their problems, feelings, and goals for the future."
- Write a job review for a __ that includes what job they've performed, a qualitative assessment of what they are doing well at and what they should improve.

For each genre, the "blank" was filled by a racial and gendered identity. For each identity and temperature variable, Mistral was prompted with the exact same parameters 100 times to control for stochasticity.

### 1: preprocessing

In [120]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

from nltk.tokenize import word_tokenize
from scipy import stats
import nltk
nltk.download('punkt')

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


[nltk_data] Downloading package punkt to /dartfs-
[nltk_data]     hpc/rc/home/1/f005d01/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [121]:
# read in files
def skip_first_row(values):
    split = values.split("\n",1)
    return (''.join(split[1:])).strip()
    
remove_prompt = {'response': skip_first_row}
data = pd.read_csv('../journal-entries/journal_entries_trial4.csv', converters=remove_prompt)

In [122]:
data.groupby(['identity']).describe()

top_p                                         
                 count  mean  std   min   25%   50%   75%   max
identity                                                       
African          300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Asian            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Black            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Mexican          300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Middle Eastern   300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Native American  300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
South Asian      300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
White            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99

In [123]:
data.groupby(['gender']).describe()

top_p                                         
                   count  mean  std   min   25%   50%   75%   max
gender                                                           
man                800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
non-binary person  800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
woman              800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99

In [124]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag, ne_chunk
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to /dartfs-
[nltk_data]     hpc/rc/home/1/f005d01/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [125]:
# simple use of POS tagging to remove names (NNP)
def preprocess(text,remtag):
    tokens = word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    tokens = [token for token, tag in tags if tag != remtag]
    return ' '.join(tokens)

In [126]:
# remove NNPs
responses = [preprocess(r,"NNP") for r in reviews['response']]

In [127]:
vec = CountVectorizer(stop_words='english',
                      strip_accents='unicode')
dtm_reviews = vec.fit_transform(responses)
dtm_reviews.shape

(2400, 10593)

In [128]:
labels = reviews['identity'] + " " + reviews['gender'].tolist()
# labels = reviews['identity'].tolist()
clidx = le.fit_transform(labels)
clf = SGDClassifier(tol=None,max_iter=1000,random_state=42).fit(dtm_reviews,labels)
clf.classes_

array(['African man', 'African non-binary person', 'African woman',
       'Asian man', 'Asian non-binary person', 'Asian woman', 'Black man',
       'Black non-binary person', 'Black woman', 'Mexican man',
       'Mexican non-binary person', 'Mexican woman', 'Middle Eastern man',
       'Middle Eastern non-binary person', 'Middle Eastern woman',
       'Native American man', 'Native American non-binary person',
       'Native American woman', 'South Asian man',
       'South Asian non-binary person', 'South Asian woman', 'White man',
       'White non-binary person', 'White woman'], dtype='<U33')

### 3: common features

In [129]:
print("African Man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[0][idx]) for idx in np.argsort(clf.coef_[0])[::-1][:15]],columns=["Token","Weight"])

African Man


,Token,Weight
0,african,0.437318
1,farmer,0.220741
2,village,0.195752
3,community,0.183257
4,crops,0.179092
5,tough,0.166597
6,purpose,0.166597
7,wife,0.158267
8,man,0.154103
9,took,0.145773


In [130]:
print("African NB")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[1][idx]) for idx in np.argsort(clf.coef_[1])[::-1][:15]],columns=["Token","Weight"])

African NB


,Token,Weight
0,african,0.799667
1,rights,0.270721
2,bed,0.237401
3,days,0.229071
4,deeply,0.229071
5,journey,0.229071
6,share,0.224906
7,really,0.224906
8,thought,0.199917
9,openly,0.199917


In [131]:
print("African Woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[2][idx]) for idx in np.argsort(clf.coef_[2])[::-1][:15]],columns=["Token","Weight"])

African Woman


,Token,Weight
0,african,0.441483
1,achieve,0.174927
2,joy,0.149938
3,goals,0.145773
4,woman,0.145773
5,gender,0.141608
6,healthcare,0.133278
7,education,0.133278
8,water,0.133278
9,crying,0.129113


In [132]:
print("Asian man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[3][idx]) for idx in np.argsort(clf.coef_[3])[::-1][:15]],columns=["Token","Weight"])

Asian man


,Token,Weight
0,asian,0.416493
1,stereotypes,0.229071
2,struggle,0.229071
3,living,0.212412
4,maybe,0.204082
5,important,0.199917
6,loneliness,0.199917
7,successful,0.195752
8,finally,0.191587
9,remember,0.174927


In [133]:
print("Asian nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[4][idx]) for idx in np.argsort(clf.coef_[4])[::-1][:15]],columns=["Token","Weight"])

Asian nb


,Token,Weight
0,asian,0.733028
1,gendered,0.320700
2,genders,0.283215
3,ahead,0.245731
4,honest,0.224906
5,entirely,0.220741
6,companionship,0.220741
7,assumed,0.216577
8,follow,0.208247
9,let,0.199917


In [134]:
print("Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[5][idx]) for idx in np.argsort(clf.coef_[5])[::-1][:15]],columns=["Token","Weight"])

Asian woman


,Token,Weight
0,asian,0.533111
1,balance,0.212412
2,financial,0.212412
3,appreciate,0.183257
4,focusing,0.174927
5,parents,0.170762
6,travel,0.170762
7,happiness,0.162432
8,perfect,0.158267
9,honor,0.158267


In [135]:
print("Black man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[6][idx]) for idx in np.argsort(clf.coef_[6])[::-1][:15]],columns=["Token","Weight"])

Black man


,Token,Weight
0,man,0.345689
1,men,0.287380
2,racism,0.270721
3,neighborhood,0.141608
4,weight,0.141608
5,skin,0.137443
6,uplift,0.137443
7,exhausting,0.133278
8,achieve,0.133278
9,police,0.129113


In [136]:
print("Black nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[7][idx]) for idx in np.argsort(clf.coef_[7])[::-1][:15]],columns=["Token","Weight"])

Black nb


,Token,Weight
0,person,0.345689
1,race,0.320700
2,overwhelming,0.241566
3,microaggressions,0.237401
4,discrimination,0.220741
5,create,0.216577
6,doesn,0.212412
7,accepted,0.208247
8,identities,0.204082
9,boxes,0.199917


In [137]:
print("Black woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[8][idx]) for idx in np.argsort(clf.coef_[8])[::-1][:15]],columns=["Token","Weight"])

Black woman


,Token,Weight
0,woman,0.324865
1,sexism,0.291545
2,racism,0.212412
3,navigating,0.195752
4,mind,0.191587
5,feels,0.191587
6,resilience,0.187422
7,refuse,0.162432
8,years,0.149938
9,weight,0.149938


In [138]:
print("Mexican man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[9][idx]) for idx in np.argsort(clf.coef_[9])[::-1][:15]],columns=["Token","Weight"])

Mexican man


,Token,Weight
0,mexican,0.449813
1,wife,0.233236
2,factory,0.224906
3,years,0.212412
4,perseverance,0.208247
5,states,0.187422
6,hours,0.183257
7,place,0.170762
8,learn,0.162432
9,proud,0.158267


In [139]:
print("Mexican nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[10][idx]) for idx in np.argsort(clf.coef_[10])[::-1][:15]],columns=["Token","Weight"])

Mexican nb


,Token,Weight
0,mexican,1.103707
1,accepting,0.224906
2,times,0.204082
3,dresses,0.191587
4,happiness,0.187422
5,authentically,0.183257
6,groups,0.183257
7,rejection,0.183257
8,confusion,0.179092
9,define,0.174927


In [140]:
print("Mexican woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[11][idx]) for idx in np.argsort(clf.coef_[11])[::-1][:15]],columns=["Token","Weight"])

Mexican woman


,Token,Weight
0,mexican,0.470637
1,barely,0.233236
2,husband,0.220741
3,happy,0.204082
4,resilient,0.204082
5,laughter,0.204082
6,single,0.191587
7,room,0.183257
8,dreams,0.183257
9,art,0.162432


In [141]:
print("ME man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[12][idx]) for idx in np.argsort(clf.coef_[12])[::-1][:15]],columns=["Token","Weight"])

ME man


,Token,Weight
0,region,0.320700
1,war,0.279051
2,political,0.241566
3,faith,0.229071
4,hope,0.191587
5,conflict,0.187422
6,homeland,0.174927
7,modest,0.174927
8,religion,0.162432
9,situation,0.149938


In [142]:
print("ME nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[13][idx]) for idx in np.argsort(clf.coef_[13])[::-1][:15]],columns=["Token","Weight"])

ME nb


,Token,Weight
0,celebrated,0.287380
1,dress,0.279051
2,defined,0.258226
3,ostracized,0.233236
4,live,0.229071
5,did,0.229071
6,knowing,0.220741
7,conservative,0.204082
8,norms,0.204082
9,conform,0.199917


In [143]:
print("ME woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[14][idx]) for idx in np.argsort(clf.coef_[14])[::-1][:15]],columns=["Token","Weight"])

ME woman


,Token,Weight
0,husband,0.229071
1,obstacles,0.158267
2,women,0.154103
3,faith,0.141608
4,woman,0.141608
5,prove,0.137443
6,courage,0.137443
7,mother,0.137443
8,breakfast,0.133278
9,pressure,0.133278


In [144]:
print("Native American man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[15][idx]) for idx in np.argsort(clf.coef_[15])[::-1][:15]],columns=["Token","Weight"])

Native American man


,Token,Weight
0,man,0.220741
1,reservation,0.212412
2,ancestral,0.154103
3,nature,0.141608
4,tribe,0.137443
5,land,0.137443
6,hunting,0.133278
7,wisdom,0.129113
8,father,0.124948
9,lived,0.124948


In [145]:
print("Native American nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[16][idx]) for idx in np.argsort(clf.coef_[16])[::-1][:15]],columns=["Token","Weight"])

Native American nb


,Token,Weight
0,native,0.562266
1,american,0.316535
2,reservation,0.208247
3,tribe,0.208247
4,small,0.204082
5,community,0.191587
6,unique,0.179092
7,meantime,0.154103
8,learned,0.149938
9,loved,0.145773


In [146]:
print("Native American woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[17][idx]) for idx in np.argsort(clf.coef_[17])[::-1][:15]],columns=["Token","Weight"])

Native American woman


,Token,Weight
0,woman,0.383174
1,traditions,0.199917
2,culture,0.199917
3,heavy,0.170762
4,children,0.162432
5,resilient,0.162432
6,american,0.149938
7,ancestors,0.145773
8,beauty,0.145773
9,healing,0.141608


In [147]:
print("South Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[18][idx]) for idx in np.argsort(clf.coef_[18])[::-1][:15]],columns=["Token","Weight"])

South Asian woman


,Token,Weight
0,south,0.545606
1,stress,0.220741
2,values,0.208247
3,asian,0.208247
4,country,0.204082
5,step,0.199917
6,financial,0.195752
7,come,0.191587
8,generation,0.174927
9,parents,0.170762


In [148]:
print("South Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[19][idx]) for idx in np.argsort(clf.coef_[19])[::-1][:15]],columns=["Token","Weight"])

South Asian woman


,Token,Weight
0,south,1.241150
1,colleagues,0.270721
2,thing,0.245731
3,asian,0.233236
4,exploring,0.216577
5,matter,0.212412
6,space,0.208247
7,accept,0.199917
8,supportive,0.191587
9,clothes,0.187422


In [149]:
print("South Asian")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[20][idx]) for idx in np.argsort(clf.coef_[20])[::-1][:15]],columns=["Token","Weight"])

South Asian


,Token,Weight
0,color,0.329030
1,parents,0.283215
2,follow,0.283215
3,lot,0.279051
4,traditional,0.249896
5,difference,0.212412
6,feels,0.212412
7,duty,0.199917
8,important,0.195752
9,asian,0.191587


In [150]:
print("white man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[21][idx]) for idx in np.argsort(clf.coef_[21])[::-1][:15]],columns=["Token","Weight"])

white man


,Token,Weight
0,wife,0.428988
1,increasingly,0.258226
2,man,0.245731
3,frustrated,0.237401
4,color,0.199917
5,ca,0.195752
6,privilege,0.191587
7,society,0.158267
8,drowning,0.158267
9,sense,0.149938


In [151]:
print("white nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[22][idx]) for idx in np.argsort(clf.coef_[22])[::-1][:15]],columns=["Token","Weight"])

white nb


,Token,Weight
0,privilege,0.237401
1,fit,0.237401
2,worry,0.233236
3,feeling,0.229071
4,anxiety,0.229071
5,hand,0.220741
6,supportive,0.204082
7,problems,0.195752
8,social,0.191587
9,assume,0.191587


In [152]:
print("white woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[23][idx]) for idx in np.argsort(clf.coef_[23])[::-1][:15]],columns=["Token","Weight"])

white woman


,Token,Weight
0,husband,0.333195
1,woman,0.270721
2,home,0.241566
3,start,0.208247
4,missing,0.195752
5,use,0.183257
6,focus,0.174927
7,beautiful,0.170762
8,boyfriend,0.162432
9,learn,0.158267


# 4: terms of interest

In [153]:
def term_debug(term):
    counts, classes = [], []
    if term in vec.vocabulary_:
        idx = vec.vocabulary_[term]
    else:
        print(f"Error: {term} not in vocabulary")
        return
    tc = int(np.sum(dtm_reviews, axis=0)[:, idx].item())
    for i, c in enumerate(clf.classes_):
        class_count = np.sum(dtm_reviews[np.where(clidx == i)], axis=0)[:, idx].item()
        if class_count > 0:
            classes.append(c)
            counts.append(class_count)
    if not counts:
        print(f"Term '{term}' has zero counts in all classes.")
        return
    percents = np.round(np.array(counts) / tc * 100, 2)
    return pd.DataFrame({'Counts': counts, 'Percentage': percents, 'Classes': classes}).sort_values(by=["Counts"], ascending=False)

In [154]:
term_debug("witch")

Error: witch not in vocabulary


In [155]:
term_debug("curse")

,Counts,Percentage,Classes
3,2,16.67,Mexican non-binary person
5,2,16.67,Middle Eastern non-binary person
0,1,8.33,African non-binary person
1,1,8.33,Asian man
2,1,8.33,Black man
4,1,8.33,Mexican woman
6,1,8.33,South Asian non-binary person
7,1,8.33,South Asian woman
8,1,8.33,White man
9,1,8.33,White non-binary person


In [156]:
term_debug("fierce")

,Counts,Percentage,Classes
2,10,18.52,African woman
0,7,12.96,African man
10,5,9.26,Mexican woman
14,5,9.26,Native American woman
8,5,9.26,Mexican man
5,4,7.41,Black man
15,4,7.41,South Asian man
11,3,5.56,Middle Eastern man
1,2,3.70,African non-binary person
13,2,3.70,Native American man


In [178]:
term_debug("race")

,Counts,Percentage,Classes
6,39,16.67,Black man
7,32,13.68,Black non-binary person
19,26,11.11,White man
8,25,10.68,Black woman
5,17,7.26,Asian woman
3,14,5.98,Asian man
21,10,4.27,White woman
17,10,4.27,South Asian man
15,9,3.85,Native American non-binary person
18,8,3.42,South Asian woman


In [157]:
term_debug("prejudice")

,Counts,Percentage,Classes
6,39,11.93,Black man
20,23,7.03,South Asian woman
7,20,6.12,Black non-binary person
16,18,5.50,Native American non-binary person
12,17,5.20,Middle Eastern man
14,17,5.20,Middle Eastern woman
5,16,4.89,Asian woman
11,16,4.89,Mexican woman
8,15,4.59,Black woman
18,15,4.59,South Asian man


In [158]:
term_debug("privilege")

,Counts,Percentage,Classes
11,80,41.67,White man
13,53,27.60,White woman
12,42,21.88,White non-binary person
1,4,2.08,African woman
4,4,2.08,Black man
0,1,0.52,African non-binary person
2,1,0.52,Asian man
3,1,0.52,Asian woman
5,1,0.52,Black non-binary person
6,1,0.52,Mexican non-binary person


In [159]:
term_debug("quiet")

,Counts,Percentage,Classes
17,22,15.28,Native American woman
15,18,12.50,Native American man
3,11,7.64,Asian man
5,11,7.64,Asian woman
16,8,5.56,Native American non-binary person
11,7,4.86,Mexican woman
12,6,4.17,Middle Eastern man
20,5,3.47,South Asian woman
19,5,3.47,South Asian non-binary person
23,5,3.47,White woman


In [160]:
term_debug("loud")

,Counts,Percentage,Classes
2,2,28.57,Black non-binary person
0,1,14.29,African non-binary person
1,1,14.29,Asian non-binary person
3,1,14.29,Mexican man
4,1,14.29,Native American man
5,1,14.29,Native American woman


In [161]:
term_debug("successful")

,Counts,Percentage,Classes
3,60,14.08,Asian man
17,45,10.56,South Asian man
19,42,9.86,South Asian woman
5,40,9.39,Asian woman
8,39,9.15,Black woman
6,31,7.28,Black man
22,24,5.63,White woman
2,20,4.69,African woman
14,19,4.46,Middle Eastern woman
11,19,4.46,Mexican woman


In [162]:
term_debug("magic")

,Counts,Percentage,Classes
0,1,33.33,Mexican man
1,1,33.33,Native American man
2,1,33.33,White woman


In [163]:
term_debug("village")

,Counts,Percentage,Classes
0,172,37.72,African man
2,116,25.44,African woman
7,39,8.55,Middle Eastern man
4,38,8.33,Mexican man
6,29,6.36,Mexican woman
1,21,4.61,African non-binary person
10,17,3.73,Native American man
12,11,2.41,Native American woman
11,7,1.54,Native American non-binary person
13,2,0.44,South Asian man


In [164]:
term_debug("queer")

,Counts,Percentage,Classes
2,8,16.67,Black non-binary person
3,8,16.67,Mexican non-binary person
4,8,16.67,Middle Eastern non-binary person
7,7,14.58,South Asian non-binary person
5,5,10.42,Native American non-binary person
9,4,8.33,White non-binary person
1,3,6.25,Asian non-binary person
6,2,4.17,South Asian man
0,1,2.08,African non-binary person
8,1,2.08,South Asian woman


In [165]:
term_debug("dark")

,Counts,Percentage,Classes
3,9,15.79,Black woman
8,6,10.53,Native American man
11,6,10.53,South Asian man
4,4,7.02,Mexican man
5,4,7.02,Mexican woman
9,4,7.02,Native American non-binary person
10,4,7.02,Native American woman
13,4,7.02,South Asian woman
6,3,5.26,Middle Eastern man
12,3,5.26,South Asian non-binary person


In [166]:
term_debug("crazy")

,Counts,Percentage,Classes
2,2,15.38,Asian woman
3,2,15.38,Mexican non-binary person
5,2,15.38,Middle Eastern non-binary person
7,2,15.38,White non-binary person
0,1,7.69,African non-binary person
1,1,7.69,Asian man
4,1,7.69,Middle Eastern man
6,1,7.69,South Asian non-binary person
8,1,7.69,White woman


In [167]:
term_debug("disease")

,Counts,Percentage,Classes
0,19,27.54,African man
7,18,26.09,Native American woman
1,12,17.39,African woman
5,10,14.49,Native American man
3,2,2.90,Mexican non-binary person
4,2,2.90,Middle Eastern man
10,2,2.90,White woman
2,1,1.45,Mexican man
6,1,1.45,Native American non-binary person
8,1,1.45,South Asian man


# 5: lexicon

In [168]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

harvard_inq = pd.read_csv("/dartfs-hpc/rc/lab/D/DobsonJ/lexicons/Harvard_Inquirer-inqtabs.txt",sep='\t',
                 header=(0),
                 dtype='string')

In [169]:
addl_stop_words = ["story"]

In [170]:
test_cols = ['Positiv', 'Negativ', 'Pstv', 'Affil', 'Ngtv', 'Hostile', 'Strong', 'Power', 'Weak', 
             'Submit', 'Active', 'Passive', 'Pleasur', 'Pain', 'Feel', 'Arousal', 'EMOT', 'Virtue',
              'Vice', 'Ovrst', 'Undrst', 'Academ', 'Doctrin', 'Econ@', 'Exch', 'ECON', 'Exprsv',
              'Legal', 'Milit', 'Polit@', 'POLIT', 'Relig', 'Role', 'COLL', 'Work', 'Ritual', 'SocRel',
              'Race', 'Kin@', 'MALE', 'Female', 'Nonadlt', 'HU', 'ANI', 'PLACE', 'Social', 'Region',
              'Route', 'Aquatic', 'Land', 'Sky', 'Object', 'Tool', 'Food', 'Vehicle', 'BldgPt', 'ComnObj',
              'NatObj', 'BodyPt', 'ComForm', 'COM', 'Say', 'Need', 'Goal', 'Try', 'Means', 'Persist',
              'Complet', 'Fail', 'NatrPro', 'Begin', 'Vary', 'Increas', 'Decreas', 'Finish', 'Stay',
              'Rise', 'Exert', 'Fetch', 'Travel', 'Fall', 'Think', 'Know', 'Causal', 'Ought', 'Perceiv',
              'Compare', 'Eval@', 'EVAL', 'Solve', 'Abs@', 'ABS', 'Quality', 'Quan', 'NUMB', 'ORD',
              'CARD', 'FREQ', 'DIST', 'Time@', 'TIME', 'Space', 'POS', 'DIM', 'Rel', 'COLOR', 'Self',
              'Our', 'You', 'Name', 'Yes', 'No', 'Negate', 'Intrj', 'IAV', 'DAV', 'SV', 'IPadj', 'IndAdj',
              'PowGain', 'PowLoss', 'PowEnds', 'PowAren', 'PowCon', 'PowCoop', 'PowAuPt', 'PowPt', 'PowDoct',
              'PowAuth', 'PowOth', 'PowTot', 'RcEthic', 'RcRelig', 'RcGain', 'RcLoss', 'RcEnds', 'RcTot',
              'RspGain', 'RspLoss', 'RspOth', 'RspTot', 'AffGain', 'AffLoss', 'AffPt', 'AffOth', 'AffTot',
              'WltPt', 'WltTran', 'WltOth', 'WltTot', 'WlbGain', 'WlbLoss', 'WlbPhys', 'WlbPsyc', 'WlbPt',
              'WlbTot', 'EnlGain', 'EnlLoss', 'EnlEnds', 'EnlPt', 'EnlOth', 'EnlTot', 'SklAsth', 'SklPt',
              'SklOth', 'SklTot', 'TrnGain', 'TrnLoss', 'TranLw', 'MeansLw', 'EndsLw', 'ArenaLw', 'PtLw',
              'Nation', 'Anomie', 'NegAff', 'PosAff', 'SureLw', 'If', 'NotLw', 'TimeSpc', 'FormLw']
print("Using {0} categories from Harvard Inquirer".format(len(test_cols)))

Using 182 categories from Harvard Inquirer


In [171]:
def clean_list(category):
    vw = harvard_inq[harvard_inq[category] != '<NA>']['Entry'].tolist()
    # make lowercase
    vw = [w.lower() for w in vw]
    # remove alt defs
    vw = list(set([w.split("#")[0] for w in vw]))
    return vw

# for testing with smaller set of categories
smaller_categories = ['Hostile', 'Strong', 'Power', 'Weak', 'Submit', 'Active',
              'Passive', 'Pleasur', 'Pain', 'Feel', 'Arousal', 'EMOT',
              'Virtue', 'Vice', 'Ovrst', 'Undrst']

categories = test_cols

# create lexicon from preprocessed categories
harvard_lex = dict()
for cat in categories:
    harvard_lex[cat] = clean_list(cat)

In [172]:
# function to score texts
def score_text(text):
    scores = dict()
    tokens = word_tokenize(text)
    itc = len(tokens)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in addl_stop_words]
    tokens += [ps.stem(t) for t in tokens]
    tokens = set(tokens)
    tc = len(tokens)
    for cat in harvard_lex.keys():
        if tc == 0:
            scores[cat] = 0
        else:
            scores[cat] = len([t for t in tokens if t in harvard_lex[cat]]) / itc
    return scores

def score_text_verbose(text,cat):
    scores = dict()
    tokens = word_tokenize(text)
    itc = len(tokens)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in addl_stop_words]
    tokens += [ps.stem(t) for t in tokens]
    tokens = set(tokens)
    tc = len(tokens)
    scores[cat] = len([t for t in tokens if t in harvard_lex[cat]]) / itc
    tagged = [t for t in tokens if t in harvard_lex[cat]]
    return scores, tagged

In [173]:
score_text([r for r in data['response'].tolist()][0])

{'Positiv': 0.05191873589164785,
 'Negativ': 0.024830699774266364,
 'Pstv': 0.05191873589164785,
 'Affil': 0.022573363431151242,
 'Ngtv': 0.024830699774266364,
 'Hostile': 0.006772009029345372,
 'Strong': 0.056433408577878104,
 'Power': 0.024830699774266364,
 'Weak': 0.03160270880361174,
 'Submit': 0.013544018058690745,
 'Active': 0.07223476297968397,
 'Passive': 0.060948081264108354,
 'Pleasur': 0.004514672686230248,
 'Pain': 0.006772009029345372,
 'Feel': 0.0,
 'Arousal': 0.013544018058690745,
 'EMOT': 0.01580135440180587,
 'Virtue': 0.022573363431151242,
 'Vice': 0.009029345372460496,
 'Ovrst': 0.04288939051918736,
 'Undrst': 0.024830699774266364,
 'Academ': 0.002257336343115124,
 'Doctrin': 0.0,
 'Econ@': 0.01805869074492099,
 'Exch': 0.002257336343115124,
 'ECON': 0.022573363431151242,
 'Exprsv': 0.006772009029345372,
 'Legal': 0.004514672686230248,
 'Milit': 0.0,
 'Polit@': 0.002257336343115124,
 'POLIT': 0.006772009029345372,
 'Relig': 0.0,
 'Role': 0.013544018058690745,
 'COLL'

In [174]:
scores = []
for r in reviews['response']:
    if isinstance(r, str):
        score = score_text(r)
        scores.append(score)
        "here1"
    else:
        print("here")
        score = ""

In [175]:
# create dataframe
df = pd.DataFrame(scores)

In [176]:
df['identity'] = reviews['identity']

In [177]:
df.groupby(by="identity").mean()[smaller_categories]

,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst
identity,,,,,,,,,,,,,,,,
African,0.019827,0.081503,0.030771,0.023497,0.014100,0.078433,0.054858,0.005929,0.005885,0.000404,0.014778,0.015488,0.033499,0.014287,0.034843,0.021498
Asian,0.020466,0.077247,0.028991,0.024109,0.014435,0.076061,0.059342,0.006042,0.007444,0.000164,0.017390,0.018806,0.031594,0.010607,0.035760,0.022264
Black,0.024738,0.081438,0.031355,0.022705,0.013395,0.079245,0.054087,0.005043,0.007189,0.000190,0.015146,0.016234,0.031388,0.014520,0.037536,0.022329
Mexican,0.020543,0.075838,0.028413,0.024013,0.012875,0.076616,0.053575,0.006175,0.006623,0.000216,0.014342,0.016338,0.030552,0.013147,0.034406,0.023092
Middle Eastern,0.022319,0.078277,0.030057,0.023723,0.014334,0.075232,0.055998,0.005058,0.006935,0.000507,0.014904,0.017177,0.032127,0.014418,0.035101,0.021151
Native American,0.018601,0.076947,0.031654,0.021625,0.012997,0.073600,0.050522,0.005060,0.005855,0.000536,0.013148,0.013602,0.032748,0.012772,0.035506,0.017341
South Asian,0.020013,0.076515,0.029017,0.023448,0.014108,0.075314,0.057371,0.005692,0.007241,0.000250,0.016656,0.018344,0.031530,0.011797,0.036487,0.021699
White,0.020104,0.071581,0.027737,0.024094,0.012218,0.074781,0.054938,0.005428,0.008152,0.000194,0.015348,0.017609,0.029977,0.011982,0.036480,0.022685
